In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [2]:
# # Create embeddingsclear
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

db = Chroma(persist_directory="./db-sales-enquiry",
            embedding_function=embeddings)

# # Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

C:\Users\PWaghma3\AppData\Local\Temp\ipykernel_21644\2831233928.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(persist_directory="./db-sales-enquiry",


In [7]:
# # Create Ollama language model - Gemma 2
local_llm = 'phi3:latest'

llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=512,  
                 temperature=0)


In [8]:
# Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based only on the following context and provide a meaningful response. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the answer, politely state that you don't have the specific information and suggest contacting the sales department for more details.

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

# Create the RAG chain using LCEL with prompt printing and streaming output
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [7]:
response = rag_chain.invoke('RfP location is grouped incorrectly')
response.content

"The issue with the incorrect grouping of an RFp (Rebate for Fixed Price) sales location seems to be related to account enrollment in Alliance, as indicated by multiple documents. To resolve this problem and ensure accurate rebates through Alliance, it is recommended that you have your account sign up on the Acuvue Alliance portal or reach out directly to SA via viseyesite@its.jnj.com for manual enrollment if necessary. Additionally, please verify with New Accounts regarding any updates needed in payer/RfP grouping under myAPP and ensure that all check-related issues are addressed by contacting the respective GPO headquarters or referring to the CX team as appropriate based on your account's affiliation status (Alliance groupings align through a common indirect customer reference account number)."

In [10]:

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": prompt}
)

In [11]:
response = qa_chain.invoke('RfP location is grouped incorrectly')
response.content

KeyboardInterrupt: 